In [31]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [32]:
data = pd.read_csv("hotel_bookings.csv")

Melyik oszlop hány százalékben található null érték

In [33]:
data.isnull().sum() / data.shape[0]

hotel                             0.000000
is_canceled                       0.000000
lead_time                         0.000000
arrival_date_year                 0.000000
arrival_date_month                0.000000
arrival_date_week_number          0.000000
arrival_date_day_of_month         0.000000
stays_in_weekend_nights           0.000000
stays_in_week_nights              0.000000
adults                            0.000000
children                          0.000034
babies                            0.000000
meal                              0.000000
country                           0.004087
market_segment                    0.000000
distribution_channel              0.000000
is_repeated_guest                 0.000000
previous_cancellations            0.000000
previous_bookings_not_canceled    0.000000
reserved_room_type                0.000000
assigned_room_type                0.000000
booking_changes                   0.000000
deposit_type                      0.000000
agent      

A children, country, agent és company oszlopokban van null érték.
Itt több lehetőség is van, hogy mit csináljunk az adott oszlopokkal.
- Ezeket az oszlopokat amikben null érték van egyszerűen csak töröljük.
- Behelyettesítjük az átlagos/medián/módusz értékével az adott oszlopnak.
- Nem csinálunk vele semmit és hagyjuk úgy ahogy vannak.

Azonban ezek közül a megoldások nem lehet egyértelműen választani, hogy melyik adatkészlethez melyik jó anélkül, hogy megvizsgálnánk ezeket az oszlopokat és elgondolkozzunk, hogy ahol hiányzik érték milyen okból kifolyólag hiányozhat.

Például: Ha egy olyan adatkészletünk van ami házaknak az árát becsüli meg abból, hogy melyik szoba mekkora, hány szoba van, van-e garázs és hiányzik néhány sorban arról adat, hogy mekkora a nappali akkor rossz döntés az, hogy az egész oszlopot töröljük, hiszen ez esetben rengeteg értékes adatot szedünk ki amivel a jövőben a modellünk pontossága csökkenhet. Egy ilyen esetben mivel általában minden háznak van nappalija elképzelhető, hogy az adatok elvesztek út közben valahol még elért hozzánk, ezért célszerű lehet a hiányzó adatokat az oszlop átlagával/medián/móduszával behelyettesíteni. Ha azonban az adatoknak kevesebb mint 1%-nál nem található csak, hogy mekkora a nappali méret, feltételezhetjük, hogy ezeknek a házaknak valóban nincs nappalijük és úgy hagyjuk ahogy vannak.

## A mi null adatainkkal mi a teendő?

### children(0.0034%) oszlop vizsgálata

In [34]:
data["children"].describe()

count    119386.000000
mean          0.103890
std           0.398561
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          10.000000
Name: children, dtype: float64

A gyerekek számának átlaga 0.10 és ha a gyerek számát növekvő sorrendbe rendezzük akkor is láthatjuk, hogy az első 75%-ban 0 gyerekkel foglaltak szállást. Így a children oszlop null értékeit behelyettesíthetjük 0-val, ugyanis elég valószínű, hogyha ez az adat hiányzik akkor a szállást foglalok nem vittek gyereket, valamint ha átlag, medián vagy módusz  alapján helyetesítenénk be ezt az értéket akkor is 0-val kéne.

In [35]:
data["children"] = data["children"].fillna(0)

### country(0.4087%) oszlop vizsgálata

In [36]:
data["country"].value_counts() / len(data)

PRT    0.406986
GBR    0.101591
FRA    0.087235
ESP    0.071765
DEU    0.061035
         ...   
DMA    0.000008
SDN    0.000008
GUY    0.000008
BDI    0.000008
MRT    0.000008
Name: country, Length: 177, dtype: float64

Mint láthatjuk itt 177 különböző kategorikus adat van. Azonban ennek a 72.86%-át az első 5 leggyakrabban jelenlévő ország teszi ki. Így a null értékek mellett rengeteg olyan értékünk is van ami maximum egyszer-kétszer jelenik meg az adathalmazban. Ezek az értékek sokkal nem segítik a modell pontosságát, hiszen maximum egy-kettő olyan sort fognak látni amit egy adott országból foglaltak. Így gondolhatnánk azt is hogy mivel a 177 különböző országból 5 teszi ki a foglalások nagy részét, hogy ezt az oszlopot csak simán el lehet hagyni, hiszen az országok többségéről nem fog sok mindent megtudni a modell abból a pár sorból amiben egy adott nemzet szerepel. Én viszont azt a lehetőséget választanám, hogy az első 5 országot úgy hagyjuk ahogy vannak, a többit és ahol null értékek vannak pedig egy "other" értékre cserélném így a modellünk nem fog olyan országot látni ami csak pár sorba szerepel. A null értékeket azért lehet biztonságosan other-re cserélni ugyanis egy adott vendégnek a világ valamelyik országából mindenféleképpen érkeznie kell.

In [37]:
data["country"] = data["country"].fillna("Other") #Null értékek feltöltése other-el

In [40]:
for index, row in data.iterrows():
    if row["country"] not in ["PRT", "GBR", "FRA", "ESP", "DEU", "Other"]:
        data.loc[:, ("country", index)] = "Other"

In [41]:
data["country"].value_counts() / len(data)

PRT      0.406986
Other    0.271388
GBR      0.101591
FRA      0.087235
ESP      0.071765
DEU      0.061035
Name: country, dtype: float64

## agent(13.68%), company(94.30%) oszlopok vizsgálata 

Ez a két oszlop vélemény szerint közel áll egymáshoz ezért lehet a két oszlopot egyszerre vizsgálni azzal kapcsolatban, hogy mit csináljunk a null értékekkel.

- agent: Ez annak az utazási irodának az id-jét tartalmazza amelyik a foglalta a szállást. Így ha ez az érték null akkor valószínűleg nem utazási irodán keresztül lett foglalva a szállás.
- company: Ez annak a cégnek az id-jét tartalmazza amelyik foglalta a szállást vagy felelős a fizetésért. Így ha ez az érték null feltétlezhetjük a foglalás privát úton történt és nem volt szükség egy ilyen közbenjáróra.

Így null értéket egy "None" értékkel fogom helyetesíteni.

In [42]:
data["agent"] = data["agent"].fillna("None")
data["company"] = data["company"].fillna("None")